#Environment and Seed Configuration
Import required libraries and initialize random seeds to ensure deterministic behavior across all experiments.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

import numpy as np
import random

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
IMAGE_SIZE = 224
NUM_CLASSES = 102
BATCH_SIZE = 32

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

eval_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_ds = datasets.Flowers102(
    root="./data",
    split="train",
    download=True,
    transform=train_transform
)

val_ds = datasets.Flowers102(
    root="./data",
    split="val",
    download=True,
    transform=eval_transform
)

test_ds = datasets.Flowers102(
    root="./data",
    split="test",
    download=True,
    transform=eval_transform
)

100%|██████████| 345M/345M [00:17<00:00, 20.3MB/s]
100%|██████████| 502/502 [00:00<00:00, 1.87MB/s]
100%|██████████| 15.0k/15.0k [00:00<00:00, 46.8MB/s]


In [ ]:
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

#CBAM (Convolutional Block Attention Module) Definition
Implement the CBAM module to apply sequential channel and spatial attention to the network's feature maps.

In [ ]:
class CBAM(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(),
            nn.Linear(channels // reduction, channels, bias=False)
        )

        self.spatial = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)

    def forward(self, x):
        b, c, _, _ = x.size()

        avg = F.adaptive_avg_pool2d(x, 1).view(b, c)
        mx  = F.adaptive_max_pool2d(x, 1).view(b, c)
        ca = torch.sigmoid(self.mlp(avg) + self.mlp(mx)).view(b, c, 1, 1)
        x = x * ca

        avg = torch.mean(x, dim=1, keepdim=True)
        mx, _ = torch.max(x, dim=1, keepdim=True)
        sa = torch.sigmoid(self.spatial(torch.cat([avg, mx], dim=1)))
        return x * sa

#Attention-ResNet-50 Model Architecture
Define the custom ResNet-50 variant integrated with CBAM modules for the Level-3 component of the ensemble.

In [ ]:
class AttentionResNet50(nn.Module):
    def __init__(self, num_classes=102):
        super().__init__()

        base = models.resnet50(pretrained=False)

        self.stem = nn.Sequential(
            base.conv1,
            base.bn1,
            base.relu,
            base.maxpool
        )

        self.layer1 = base.layer1
        self.cbam1  = CBAM(256)

        self.layer2 = base.layer2
        self.cbam2  = CBAM(512)

        self.layer3 = base.layer3
        self.cbam3  = CBAM(1024)

        self.layer4 = base.layer4
        self.cbam4  = CBAM(2048)

        self.pool = base.avgpool
        self.fc   = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.cbam1(self.layer1(x))
        x = self.cbam2(self.layer2(x))
        x = self.cbam3(self.layer3(x))
        x = self.cbam4(self.layer4(x))
        x = self.pool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

#Loading Fine-Tuned Model Weights
Load the saved state dictionaries for both the Level-2 (Augmented) and Level-3 (Attention) models into their respective architectures.

In [ ]:
NUM_CLASSES = 102

model_l2 = models.resnet50(pretrained=False)
model_l2.fc = torch.nn.Linear(model_l2.fc.in_features, NUM_CLASSES)
model_l2.load_state_dict(torch.load("/content/level2_resnet50_aug_new_with_layer_3.pth"))
model_l2.to(device)
model_l2.eval()

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
model_l3 = AttentionResNet50(NUM_CLASSES)
model_l3.load_state_dict(torch.load("/content/level3_attention_resnet50_cbam.pth"))
model_l3.to(device)
model_l3.eval()


AttentionResNet50(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, k

#Soft-Voting Ensemble Evaluation Function
Implement the logic to combine model predictions by averaging their softmax probabilities to determine the final class label.

In [ ]:
def evaluate_ensemble(model_a, model_b, loader, alpha=0.5):
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Forward pass
            out_a = F.softmax(model_a(imgs), dim=1)
            out_b = F.softmax(model_b(imgs), dim=1)

            # Soft voting
            ensemble_out = alpha * out_a + (1 - alpha) * out_b
            preds = ensemble_out.argmax(dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return correct / total

#Final Ensemble Accuracy Assessment
Execute the ensemble evaluation on the test set and report the final integrated classification performance.

In [ ]:
ensemble_acc = evaluate_ensemble(model_l2, model_l3, test_loader)
print("Level-4 Ensemble Test Accuracy:", ensemble_acc)

Level-4 Ensemble Test Accuracy: 0.9055130915596031
